In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords  # Importa la librería stopwords de NLTK

ModuleNotFoundError: No module named 'scipy.sparse.similarity'

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Cargar el archivo JSONL
with open('Digital_Music.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [ ]:
data_df = pd.DataFrame(data)
data_df.head(10)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Nice,If i had a dollar for how many times I have pl...,[],B004RQ2IRG,B004RQ2IRG,AFUOYIZBU3MTBOLYKOJE5Z35MBDA,1618972613292,0,True
1,5.0,Excellent,awesome sound - cant wait to see them in perso...,[],B0026UZEI0,B0026UZEI0,AHGAOIZVODNHYMNCBV4DECZH42UQ,1308167525000,0,True
2,5.0,Great service,This is a great cd. Good music and plays well....,[],B0055JSYHC,B0055JSYHC,AFGEM6BXCYHUILEOA3P2ZYBEF2TA,1615838793006,0,True
3,1.0,No good,"These are not real German singers, they have a...",[],B000F9SMUQ,B000F9SMUQ,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,1405219741000,0,True
4,3.0,"Cool concept, so-so execution...",I first heard this playing in a Nagoya shop an...,[],B0049D1WVK,B0049D1WVK,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,1309029595000,0,False
5,4.0,"""L'attaque des clones"" a satisfying taste of t...","I just saw ""Star Wars: L'attaque des clones"" l...",[],B00003CXKT,B00003CXKT,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,1022191122000,1,False
6,5.0,Ronnie Dunn like I've never heard before!,"This is without a doubt my favorite CD, and I ...",[],B083QN4Y6H,B083QN4Y6H,AGYEAZK4OEYF2MSSTGJ5WNJDVZKA,1592615999887,0,True
7,5.0,A true classic seventies rock album,Sounds great,[],B08L7CJ6NM,B08L7CJ6NM,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1641330592178,0,True
8,5.0,A great collection of music,This disk is fantastic I wanted the record but...,[],B092C24W4G,B092C24W4G,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1641224888411,2,True
9,5.0,Excellent musical experience,This is a fantastic collection of music superb...,[],B01M32UHXG,B01M32UHXG,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1566575806068,0,False


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130434 entries, 0 to 130433
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             130434 non-null  float64
 1   title              130434 non-null  object 
 2   text               130434 non-null  object 
 3   images             130434 non-null  object 
 4   asin               130434 non-null  object 
 5   parent_asin        130434 non-null  object 
 6   user_id            130434 non-null  object 
 7   timestamp          130434 non-null  int64  
 8   helpful_vote       130434 non-null  int64  
 9   verified_purchase  130434 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 9.1+ MB


In [ ]:
filtered_df = data_df[data_df['verified_purchase'] == True]
selected_data = filtered_df[['rating', 'title', 'text', 'helpful_vote']]

In [ ]:
selected_data.head(20)

,rating,title,text,helpful_vote
0,5.0,Nice,If i had a dollar for how many times I have pl...,0
1,5.0,Excellent,awesome sound - cant wait to see them in perso...,0
2,5.0,Great service,This is a great cd. Good music and plays well....,0
3,1.0,No good,"These are not real German singers, they have a...",0
6,5.0,Ronnie Dunn like I've never heard before!,"This is without a doubt my favorite CD, and I ...",0
7,5.0,A true classic seventies rock album,Sounds great,0
8,5.0,A great collection of music,This disk is fantastic I wanted the record but...,2
11,5.0,Great co to buy from and i am picky,It exceeds my expectations I wish all retailer...,0
12,5.0,Astounding!,"astoundingly beautiful, haunting, moving Chora...",0
13,5.0,Five Stars,Fabulous Cd well worth purchasing...,0


In [ ]:
def clean_reviews(reviews):
  """Limpia las reseñas de texto eliminando caracteres especiales, convirtiéndolas a minúsculas y eliminando palabras vacías.

  Args:
      reviews: Una lista de reseñas de texto.

  Returns:
      Una lista de reseñas limpiadas.
  """

  clean_reviews = []
  stop_words = stopwords.words('english')  # Descarga e inicializa las stop words en inglés

  for review in reviews:
    # Convertir a minúsculas
    review_lower = review.lower()

    # Eliminar caracteres de puntuación
    review_sin_puntuacion = ''.join(c for c in review_lower if c not in string.punctuation)

    # Tokenizar el texto (convertir en una lista de palabras)
    review_tokens = review_sin_puntuacion.split()

    # Eliminar stop words
    review_clean = [palabra for palabra in review_tokens if palabra not in stop_words]

    # Unir las palabras limpias en una cadena de texto
    review_clean_str = ' '.join(review_clean)

    clean_reviews.append(review_clean_str)

  return clean_reviews

In [ ]:
# Limpiar columna numérica (eliminar NaN)
def clean_numeric_data(data):
    return data.dropna()

In [ ]:
# Aplicar la función de limpieza en base al tipo de dato
clean_df = selected_data.copy()

In [ ]:
clean_df = clean_numeric_data(clean_df)

In [ ]:
clean_df['title'] = clean_reviews(clean_df['title'].tolist())
clean_df['text'] = clean_reviews(clean_df['text'].tolist())

In [ ]:
clean_df.head(20)

,rating,title,text,helpful_vote
0,5.0,nice,dollar many times played cd many times asked a...,0
1,5.0,excellent,awesome sound cant wait see person always miss...,0
2,5.0,great service,great cd good music plays well seller responde...,0
3,1.0,good,real german singers accents nothing advertised...,0
6,5.0,ronnie dunn like ive never heard,without doubt favorite cd may close wearing en...,0
7,5.0,true classic seventies rock album,sounds great,0
8,5.0,great collection music,disk fantastic wanted record recieved couple b...,2
11,5.0,great co buy picky,exceeds expectations wish retailers great mayan,0
12,5.0,astounding,astoundingly beautiful haunting moving chorale,0
13,5.0,five stars,fabulous cd well worth purchasing,0


In [ ]:
# Crear un vectorizador TF-IDF para transformar las reseñas en vectores numéricos
tfidf_vectorizer = TfidfVectorizer()
# Transformar las reseñas en vectores TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(clean_df['text'])


In [ ]:
def recommend_music(review, vectorizer, similarity_matrix, data_df, k=5):
    # Limpiar reseña
    user_review = clean_reviews(review)
    # Convertir la reseña del usuario en un vector TF-IDF
    user_review_vector = vectorizer.transform(user_review)

    # Calcular la similitud entre la reseña del usuario y todas las demás reseñas
    user_similarities = linear_kernel(user_review_vector, similarity_matrix).flatten()

    # Identificar los índices de las k reseñas más similares
    similar_indices = user_similarities.argsort()[-k:][::-1]

    # Obtener las reseñas recomendadas
    recommended_reviews = data_df.iloc[similar_indices]

    # Devolver las recomendaciones en un formato enriquecido
    recommendations = recommended_reviews.to_dict(orient='records')

    return recommendations

In [ ]:
review = ["Me encanta esta canción con una excelente melodía"]

recommendations = recommend_music(review, tfidf_vectorizer, tfidf_matrix, clean_df, k=3)
print(recommendations)

[{'rating': 5.0, 'title': 'trae recuerdos del ayer', 'text': 'encanta', 'helpful_vote': 0}, {'rating': 5.0, 'title': 'bts life', 'text': 'encanta', 'helpful_vote': 0}, {'rating': 5.0, 'title': 'boleros', 'text': 'excelente calidad de video sonido con un señor de la canción más que excelente', 'helpful_vote': 1}]
